In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def new_positions_spherical_coordinates(num_points, radius, x_offset=0.0, y_offset=0.0, z_offset=0.0, stddev=0.0):
    radius = radius + stddev * np.random.rand()
    print(radius)
    theta = np.random.uniform(0.,2. * np.pi ,(num_points,1))
    phi = np.arccos(1-2*np.random.uniform(0.0,1.,(num_points,1)))
    x = radius * np.sin( theta ) * np.cos( phi ) + x_offset + stddev * np.random.rand()
    y = radius * np.sin( theta ) * np.sin( phi ) + y_offset + stddev * np.random.rand()
    z = radius * np.cos( theta ) + z_offset + stddev * np.random.rand()
    return (x,y,z)

In [3]:
np.set_printoptions(precision=4)

In [4]:
# Generate random point set
r = 44.0
n = 10
(x,y,z) = new_positions_spherical_coordinates(n, r,
                                              x_offset=0.0,
                                              y_offset=0.0,
                                              z_offset=0.0, 
                                              stddev=0.5)

44.1563874476


In [5]:
# Point set from Hildenbrand
# n = 5
# x = [1.0, 1.0, 0.0, 0.0, -1.0]
# y = [0.0, 1.0, 0.0, 1.0,  0.0]
# z = [0.0, 0.0, 1.0, 1.0,  1.0]

In [6]:
ps = [np.array(np.ravel(p).tolist() + 
               [1.0, 0.5*np.linalg.norm(p)**2]).reshape(5,1) 
      for p in zip(x,y,z)]

## Total Least Squares Fitting of $k$-Spheres in $n$-D Euclidean Space Using an $(n + 2)$-D Isometric Representation

Leo Dorst 2014

Journal of Mathematical Imaging and Vision

http://link.springer.com/article/10.1007%2Fs10851-014-0495-2


Cost function:
$$\frac{1}{N}\sum_i\frac{(p_i \cdot x)^2}{x^2}$$

In [7]:
M = np.zeros((5,5))
M[:3,:3] = np.eye(3)
M[3,4] = -1
M[4,3] = -1
P = np.zeros((5,5))
for p in ps:
    P += np.inner(np.outer(p,p),M)
(w,v) = np.linalg.eig(P / float(len(ps)))
idx = np.where(w == np.min(w[w>0]))[0][0]
x_ = v[:,idx]
x_ /= x_[3]
print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])
print("Estimated radius: {}".format(radius))

[  2.8169e-01   4.3111e-02   1.8332e-01   1.0000e+00  -9.7484e+02]
Estimated radius: 44.1563874476


In [8]:
# M

In [9]:
# P

## Foundations of Geometric Algebra Computing

Dietmar Hildenbrand 2013

In [47]:
P = np.zeros((5,5))
for p in ps:
    P += np.outer(p,p)
(w,v) = np.linalg.eig(P)
idx = np.where(w == np.min(w[w>0]))[0][0]
x_ = v[:,idx]
x_ /= x_[4]
print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[3])
print("Estimated radius: {}".format(radius))

[ -2.8169e-01  -4.3111e-02  -1.8332e-01  -9.7484e+02   1.0000e+00]
Estimated radius: 44.1563874476


In [11]:
# P

In [12]:
import sys
sys.path.append('../build/Debug/')
import libsphere_fit as sphere_fit

In [13]:
def create_solver_options(use_trust_region_minimizer=True):
    solver_options = {'minimizer_type':'TRUST_REGION',
                      'trust_region_strategy_type':'LEVENBERG_MARQUARDT',
                      'linear_solver_type':'DENSE_QR',
                      'max_num_iterations': 100, 
                      'num_threads': 12,
                      'num_linear_solver_threads':12,
                      'parameter_tolerance': 10e-12,
                      'function_tolerance': 10e-12,
                      'minimizer_progress_to_stdout':False,
                      'trust_region_minimizer_iterations_to_dump':[],
                      'trust_region_problem_dump_directory':'/home/lars/devel/game_ws/dump/sphere_fit',
                     }
    return solver_options    

In [52]:
sf = sphere_fit.SphereFit()

solver_options = create_solver_options()
solver_options['minimizer_progress_to_stdout'] = True
solver_options['trust_region_minimizer_iterations_to_dump'] = [1,2,3,4,5,6,7,8,9]
sf.set_solver_options(solver_options)


sphere = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(5,1)
points = np.ascontiguousarray(np.transpose([x,y,z])).reshape(n,3).copy()
x_ = sf.run(sphere, points).copy()
x_ = x_ / x_[3]
print(x_.reshape(-1))
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])[0]
print("Estimated radius: {}".format(radius))

[  2.8169e-01   4.3111e-02   1.8332e-01   1.0000e+00  -9.7484e+02]
Estimated radius: 44.1563874476


In [15]:
sphere.reshape(-1)

array([  1.0827e-02,   1.6570e-03,   7.0458e-03,   3.8435e-02,  -3.7468e+01])

In [16]:
# sf.summary()

In [17]:
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.set_aspect("equal")
# u, v = np.mgrid[0:2*np.pi:20j, 0:-np.pi:10j]
# sx= r * np.cos(u)*np.sin(v)
# sy= r * np.sin(u)*np.sin(v)
# sz= r * np.cos(v)
# ax.plot_wireframe(sx, sy, sz, color="r")

# # ax.scatter3D(x,y,z)
# sx= radius * np.cos(u)*np.sin(v) + x_[0]
# sy= radius * np.sin(u)*np.sin(v) + x_[1]
# sz= radius * np.cos(v) + x_[2]
# ax.plot_wireframe(sx, sy, sz, color="b")

In [18]:
def load_problem_iteration_dump(directory, iteration):
    tmp = '/ceres_solver_iteration_{:0>3d}'.format(iteration)
    D = np.loadtxt(directory + tmp + '_D.txt')
    b = np.loadtxt(directory + tmp + '_b.txt')
    x = np.loadtxt(directory + tmp + '_x.txt')
    f = open(directory + tmp + '.m')
    matlab_script = f.readlines()
    f.close()
    rows = [int(s) for s in matlab_script[1][:-2].split() if s.isdigit()][0]
    cols = [int(s) for s in matlab_script[2][:-2].split() if s.isdigit()][0]
    A_sparse = np.loadtxt(directory + tmp + '_A.txt')
    A = np.zeros((rows, cols))
    for row in A_sparse:
        A[int(row[0]), int(row[1])] = row[2]
    return (A,D,b,x)

directory = solver_options['trust_region_problem_dump_directory']
(A, D, b, x_) = load_problem_iteration_dump(directory, 1)
A

array([[ 0.0583,  0.0692,  0.0652, -0.0954, -0.0605],
       [ 0.0656,  0.0669,  0.0616, -0.0957, -0.0634],
       [ 0.0663,  0.0693,  0.0579, -0.0959, -0.0601],
       [ 0.0683,  0.0678,  0.0579, -0.0962, -0.0592],
       [ 0.0634,  0.0612,  0.0699, -0.0967, -0.0578],
       [ 0.0624,  0.0629,  0.0687, -0.0966, -0.0566],
       [ 0.0695,  0.0626,  0.063 , -0.0963, -0.0629],
       [ 0.0685,  0.0667,  0.0588, -0.0965, -0.0576],
       [ 0.0638,  0.0638,  0.0669, -0.0958, -0.0636],
       [ 0.0632,  0.061 ,  0.0706, -0.0964, -0.0607]])

In [45]:
D

array([ 0.002 ,  0.002 ,  0.002 ,  0.003 ,  0.0019])